In [1]:
import numpy as np
from astropy.table import QTable
import astropy.coordinates as coord
import astropy.units as u

from galpy.potential import NFWPotential, HernquistPotential, MiyamotoNagaiPotential
from galpy.orbit import Orbit

# Importing data and transforming to Galactic Coordinates

In [3]:
# importing the data as an astropy table
# the data has been downloaded with the following criteria:
# - non null radial velocities
# - parallax_over_error > 5
# - ruwe <1 .4
# - parallax > 0.183 (to account for the parallax zero point of 0.017 mas)
gaia_rad = QTable.read("1629912465318O-result.csv")

In [4]:
# getting distance for each source
dist = coord.Distance(parallax = (u.Quantity(gaia_rad['parallax'] ) +0.017)*u.mas)

In [5]:
# Making a SkyCoord object to easily transform between coordinates
c = coord.SkyCoord(ra = gaia_rad['ra']*u.deg,
                   dec = gaia_rad['dec']*u.deg,
                   distance = dist,
                   pm_ra_cosdec = gaia_rad['pmra']*u.mas/u.year,
                   pm_dec = gaia_rad['pmdec']*u.mas/u.year,
                   radial_velocity = gaia_rad['dr2_radial_velocity']*u.km/u.s,
                   frame = 'icrs')

In [6]:
orbits = Orbit(c)

# Setting up the Potential

In [7]:
G = 4.301 * 10**(-6) *u.kpc* (u.kilometer/u.s)**2/u.solMass
M_h = 10**12 *u.solMass
M_d = 9.3*10**10 *u.solMass
M_b = 3*10**10 *u.solMass
r_s = 21.5 *u.kpc 
a_d = 6.5 *u.kpc  
b_d = 0.26 *u.kpc 
c_b = 0.7 *u.kpc
   
HaloPot = NFWPotential(amp = G*M_h/(np.log(2)-0.5), a = r_s)
DiskPot = MiyamotoNagaiPotential(amp = G*M_d, a = a_d, b = b_d)
BulgePot = HernquistPotential(amp = 2*G*M_b, a = c_b)
Potential = HaloPot + DiskPot + BulgePot

# Computing Integrals of Motion

In [8]:
E = orbits.E(pot = Potential)
Lz = orbits.Lz(pot = Potential)
x = orbits.x()
y = orbits.y()
z = orbits.z()
vx = orbits.vx()
vy = orbits.vy()
vz = orbits.vz()